
## Trigonometric stuff for free
There is a thing called CORDIC, that is able to calculate trigonometric functions by only using add and shift operations, so basically free.

Good overview publication is given [here](http://www.andraka.com/files/crdcsrvy.pdf).

CORDIC is based on efficient vector rotation defined as:
$$ x_{i+1} = x_{i} - y_{i} * d * 2^{-i} $$
$$ y_{i+1} = y_{i} + x_{i} * d * 2^{-i} $$

At first i tougth that the sum of one cordic phase rotation is 90 degrees, that is false as the sum is about 100 degrees

Potential error is equal to the last element phase increase.


In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

x = 0
y = 1
gain = 1
phs = 0
for i in range(12):
    dir = 1
    x, y = x - dir*(y >> i), y + dir*(x >> i)
    
    # this is only for printing
    gain = gain * np.sqrt(1 + 2 ** (-2*i))
    phase = np.arctan(2 ** -i) * 180/np.pi
    phs += phase
    print('i={}, rotation= {:.2f} deg, sum(rotation)= {:.2f}, gain= {:.6f}'.format(i, phase, phs, gain))

i=0, rotation= 45.00 deg, sum(rotation)= 45.00, gain= 1.414214
i=1, rotation= 26.57 deg, sum(rotation)= 71.57, gain= 1.581139
i=2, rotation= 14.04 deg, sum(rotation)= 85.60, gain= 1.629801
i=3, rotation= 7.13 deg, sum(rotation)= 92.73, gain= 1.642484
i=4, rotation= 3.58 deg, sum(rotation)= 96.30, gain= 1.645689
i=5, rotation= 1.79 deg, sum(rotation)= 98.09, gain= 1.646492
i=6, rotation= 0.90 deg, sum(rotation)= 98.99, gain= 1.646693
i=7, rotation= 0.45 deg, sum(rotation)= 99.44, gain= 1.646744
i=8, rotation= 0.22 deg, sum(rotation)= 99.66, gain= 1.646756
i=9, rotation= 0.11 deg, sum(rotation)= 99.77, gain= 1.646759
i=10, rotation= 0.06 deg, sum(rotation)= 99.83, gain= 1.646760
i=11, rotation= 0.03 deg, sum(rotation)= 99.85, gain= 1.646760


What we can see here is that by using only shifts and adds we can rotate vector by up to 100 degrees. Some problems however:
* It is iterative process, where every succeeding step rotates less and less
* Gain is not 1, but approximates 1.65 - we have to correct for that.

Thing to notice is that the rotation degrees is always smaller than sum of the rotations below that step.
Meaning we can implement a binary search algorithm, lets say to rotate a input vector by 37 degrees:



In [45]:
# input vector
input = 0.5 + 0.2j
angle = np.angle(input, deg=True)
print('Inital phase: {}'.format(angle))
x = input.real
y = input.imag

phs = 0
target = 37
for i in range(12):
    dir = 1 if phs < target else -1
    c = 2 ** -i
    x, y = x - dir*(y* c), y + dir*(x * c)
    phase = np.arctan(c) * 180/np.pi
    phs += dir*phase
    print('dir= {} rotation={}'.format(dir, phs))

output = x + 1j*y
angle = np.angle(output, deg=True)
print('End phase: {}'.format(angle))

Inital phase: 21.80140948635181
dir= 1 rotation=45.0
dir= -1 rotation=18.43494882292201
dir= 1 rotation=32.471192290848485
dir= 1 rotation=39.59620863975028
dir= -1 rotation=36.01987426475293
dir= 1 rotation=37.809784872998996
dir= -1 rotation=36.91461116278792
dir= 1 rotation=37.362225333648475
dir= -1 rotation=37.13841483327994
dir= -1 rotation=37.026509156213734
dir= -1 rotation=36.97055626431993
dir= 1 rotation=36.998532716936936
End phase: 58.799942203288744


What happens when we rotate our input vector to 0 phase? Well, in that case we can store the phase shift required to get to 0 point and that gives us the original phase of the signal. Lets try:


In [48]:
# input vector
input = 0.5 + 0.2j
angle = np.angle(input, deg=True)
print('Inital phase: {}'.format(angle))
x = input.real
y = input.imag

phs = 0
for i in range(12):
    dir = 1 if y < 0 else -1
    c = 2 ** -i
    x, y = x - dir*(y* c), y + dir*(x * c)
    phase = np.arctan(c) * 180/np.pi
    phs -= dir*phase
    print('dir= {} rotation={}'.format(dir, phs))

output = x + 1j*y
angle = np.angle(output, deg=True)
print('End phase: {}'.format(angle))

Inital phase: 21.80140948635181
dir= -1 rotation=45.0
dir= 1 rotation=18.43494882292201
dir= -1 rotation=32.471192290848485
dir= 1 rotation=25.346175941946687
dir= 1 rotation=21.769841566949335
dir= -1 rotation=23.559752175195406
dir= 1 rotation=22.66457846498433
dir= 1 rotation=22.216964294123777
dir= 1 rotation=21.993153793755237
dir= 1 rotation=21.88124811668903
dir= 1 rotation=21.825295224795227
dir= 1 rotation=21.797318772178222
End phase: 0.004090714173580176


I replaced the dir function to check it the y component is below zero or not. The algorithm converges and the gives us the phase value of the input. Additionally since the y component is 0, we can read the input instantainous power from the x component (that has the parasitic CORDIC scaling included).
This is big value indeed.

Found an great implementation of CORDIC performing polar <-> rectangular conversion [here](http://code.activestate.com/recipes/576792-polar-to-rectangular-conversions-using-cordic/), take a look at to_polar() function, only 5 lines of code:

In [ ]:
def to_polar(x, y):
    'Rectangular to polar conversion using ints scaled by 100000. Angle in degrees.'
    theta = 0
    for i, adj in enumerate((4500000, 2656505, 1403624, 712502, 357633, 178991, 89517, 44761)):
        sign = 1 if y < 0 else -1
        x, y, theta = x - sign*(y >> i), y + sign*(x >> i), theta - sign*adj
    return theta, x * 60726 // 100000

Now how to turn this simple code into hardware? Some problems with this code:
* CORDIC size is hard-coded(8 iterations) as are the phase values
* It uses scaled integers instead fixed point numbers, this has to change
* Multiplication is used to control the signedness, this will not work in hardware
* Uses "tricky" multiple variable assignment (notice that these assignments actually need temporary variables)

We need to deal with fixed-point numbers and resizing in order to synthesize this code, registers will need some attentions as well.
Bottom line is that we are not going to make it with 5 lines of code, in fact if we make it under 50 lines it is good.

I would start by converting this function into class and adding options to define number of iterations, that means we need to generate the phase lookup-table as well. 

$$c = \sqrt{a^2 + b^2}$$